<a href="https://colab.research.google.com/github/ChaejinE/MyPytorch/blob/main/PyTorch_Tips_Details/k_ParamUpdate(Opt%2C_Loss%2C_Backprop%2C_etc).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

# Overview
- optimizer.zero_grad() : 이전 Step에서 각 layer 별로 계산된 gradient 값을 모두 0으로 초기화 시키는 작업
- loss.backward() : 각 layer의 파라미터에 대해 back-prop 통해 gradient 계산
- optimizer.step() : 각 layer의 파라미터와 같이 저장된 Gradient를 이용해 파라미터 업데이트

# optimizer.step() <파라미터 업데이트> 와 loss.backward() 관계
```python
criterion = nn.CrossEntropyLoss()
out = model(input)
loss = criterion(out, target)
loss.backward()
```
- **weight를 update하는 시점** : optimizer.step() 호출 시점
- 보통 loss 함수로 loss를 구하고, .backward()를 통해 optimizer.step() 진행

```python
optimizer = optim.Adam(mode.parameters(), lr=lr)
optimizer.step()
```
- optimizer 선언 및 사용법
- FrameWork 순선상 1. loss 계산 2. loss.backward()로 gradient 게산 3. optimizer.step() 으로 weight update 순서가 되어야 한다.
- model에 conv1이라는 레이어가 있다고 해보자.
- model.conv1.weight.grad에 loss에 따라 계산된 gradient가 있다.
- .layer.weight.grad에 gradient가 저장되는 시점은 loss.backward()가 실행되는 시점이다.
  - 이 때 gradinent가 계산되어 저장되는 것이다.
  - .bacward() 이후 grad 값을 출력하면 그 layer의 gradient값을 볼 수 있다.
- optimizer 객체는 model.parameters()를 통해 생성되었기 때문에 loss.backward()를 통해 .layer.eight.grad에 저장된 각 layer의 gradient는 optimizer에서 바로 접근해서 사용 가능하다.
- 따라서 optimizer와 loss.backward()는 같은 model 객체를 사용한다.
  - loss.backward()의 출력 값이 model 객체 각 layer들의 grad 멤버 변수에 저장된다.
  - 이 값을 optimizer의 입력으로 다시 사용함으로써 두 연산이 연결되어 진다.

# Gradient를 직접 zero로 셋팅하는 이유
- 어떤 weight의 Gradient를 계산했을 때 그 값이 기존 gradient를 계산한 값에 누적되기 때문이다.

In [4]:
from torch.autograd import Variable

In [5]:
x = Variable(torch.Tensor([[0]]), requires_grad=True)

for t in range(5):
  y = x.sin()
  y.backward()
  print(x.grad)

tensor([[1.]])
tensor([[2.]])
tensor([[3.]])
tensor([[4.]])
tensor([[5.]])


- 위 코드를 보면 1 씩 누적되는 것을 확인할 수 있다.
- 반면 아래코드는 x.grad를 0으로 초기화 하기 때문에 gradient가 누적되지 않는다.

In [6]:
x = Variable(torch.Tensor([[0]]), requires_grad=True)

for t in range(5):
  if x.grad is not None:
    x.grad.data.zero_()
  y = x.sin()
  y.backward()
  print(x.grad)

tensor([[1.]])
tensor([[1.]])
tensor([[1.]])
tensor([[1.]])
tensor([[1.]])


- 보통 학습 시 GPU 메모리의 한계로 인해 한번에 GPU를 통해 연산되는 데이터 양이 제한적이다.
  - 데이터가 100개면 20개씩 데이터를 분할해서 5번 나눠 학습하곤한다.
  - 20 : batch size
  - 5번 : iteration, 나눠서 학습하는 횟수
- batch size x iteration을 하면 현재 가지고 있는 데이터 전체를 대상으로 학습하게 된다.
  - 전체 학습한 단위를 epcoh이라한다.
  - 10 epoch : 100개 데이터를 10번 반복 학습
- **PyTorch에서는 기본적으로 gradient를 누적하여 합하게 된다.**
  - 두가지 전략으로 학습할 수 있다.
  - 1. iteration마다 weight를 update : gradient 누적을 막기위해 **iteration 시작 시** 이전 iteration에서 계산된 gradient를 0으로 초기화한다.

```python
for epoch in range(epohcs):
  for num_train, (train_data, taget_data) in enumerate(batches):
    optimizer.zero_grad()
    out = nn_nodel(train_data)
    loss = loss_function(out, target_data)
    loss.backward()
    loss_sum += loss.item()
    optimizer.step()
  loss_list.append(loss_sum / (num_train + 1))
```


2. epoch 마다 weight update 하는 방법 : 모든 iteration에서 계산된 gradient 누적하여 한번에 weight update. **epoch 시작 시** 계산된 gradient를 0으로 초기화

```python
for epoch in range(epohcs):
  optimizer.zero_grad()
  for num_train, (train_data, taget_data) in enumerate(batches):
    out = nn_nodel(train_data)
    loss = loss_function(out, target_data)
    loss.backward()
    loss_sum += loss.item()
  optimizer.step()
  loss_list.append(loss_sum / (num_train + 1))
```


- 위 내용들을 보면 optimizer.zero_grad()를 실행하는 시점이 달라지는 것을 알 수 있다.
- iteration 마다 weight를 update하는 방법을 많이 사용한다.

# Validation의 loss 계산 시 detach 사용에 대해
- 일부 epoch 진행 후 CUDA error : out of range 에러가 발생하는 경우가 있다.
- Train은 loss 구하고 .bacward()를 통해 backprop을 하는데, Valid dataset은 loss만 구한다.
- 위 차이로 인해 의도치 않게 PyTorch 사용 시 에러가 나곤한다.

```python
out = model(input)
criterion = nn.CrossEntropy()
loss = criterion(out, target)
loss.backward()
```
- loss backward()하지 않으면 backprop하기위한 그래프 히스토리가 loss에 계속 쌓인다.
  - 각 epoch의 loss를 구하기 위해 각 batch에서 계산된 loss를 모두 더한 뒤 평균내는 방법을 많이 사용한다.
  - backward() 하지 않고 사용하면 히스토리가 계속 쌓이면서 GPU 연산에 영향이 생겨 CUDA error가 나는 것이다.

```python
out = model(input)
criterion = nn.CrossEntropyLoss()
loss = criterion(out, target).detach()
```
- detach() 함수를 통해 그래프의 히스토리를 의도적으로 끊는 방법을 사용해 메모리 문제를 피할 수 있다.
- 이 경우 backward() 연산을 사용할 수 없다.
  - Validation dataset 연산에 유용하겠다.


# model.eval() vs torch.no_grad()
- model.eval() : .eval() 모드 사용 시 모델 내부의 모든 layer가 evaluation 모드가 된다.
  - batchnorm, dropout과 같은 기능들이 사용되지 않는다.
- torch.no_grad() : Tensor가 .no_grad() 지정이 되면 autograd 엔진에게 이 정보를 알려줘 학습에서 제외된다. 학습에서 제외되므로 Backprop에 필요한 메모리를 절약할 수 있어 연산이 빨라진다. 하지만 Backprop 할수 없으므로 Train은 불가능하다.
- model.eval()은 실제 inference를 하기 전에 모든 layer를 evaluation 모드로 변경하기 위해 사용하면 된다.
- torch.no_grad()는 특정 레이어에서 Backprop을 적용하지 않기 위해 사용하면된다.
  - 학습에서 제외할 layer가 있다면 그 layer를 위해 사용하면 된다.

- .train(), .eval() 모드에 따라 dropout 적용 결과가 달라지는 것을 알 수 있다.
- eval 모드에서는 Dropout 비활성화, input을 그대로 pass하는 역할이 된다.
- train에서는 확률값 p를 받아서 사용된다.

In [21]:
drop = torch.nn.Dropout(p=0.3)
x = torch.ones(1, 10)
x

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [22]:
drop.train()
drop(x) # pytorch docs에 의하면 output이 있으면 factor < 1 / (1-p) >가 학습시 가중치에 곱해진다고 한다. 

tensor([[1.4286, 1.4286, 0.0000, 0.0000, 0.0000, 0.0000, 1.4286, 1.4286, 1.4286,
         1.4286]])

In [18]:
drop.eval()
drop(x)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

- torch.no_grad()의 사용 케이스다.

In [14]:
x = torch.tensor([1.], requires_grad=True)

# with 구문을 이용하여 Tensor의 성분을 no_grad로 변경
with torch.no_grad():
  y = x * 2
y.requires_grad

False

In [15]:
# decorator를 이용해 Tensor의 성분을 no_grad로 변경
@torch.no_grad()
def doubler(x):
  return x * 2

z = doubler(x)
z.requires_grad

False